In [2]:

import xarray as xr
import pandas as pd
from pyclim_noresm.general_util_funcs import global_avg                             
from workflow.scripts.utils import (calculate_CI, diff_means_greater_than_varability, 
                                    masked_average, t_test_diff_sample_means,
                                   make_equal_dimmension)
import numpy as np
import yaml
import dataframe_image as dfi
from functools import partial
from scipy.stats import ttest_ind, t, bootstrap
import re

In [3]:
def create_dict(paths_ctrl, paths_exp):
    """
    Create a dictionary with the paths to the control and experiment
    """
    paths = {path.split('/')[-1].split('_')[0] : {} for path in paths_ctrl}

    for var in paths:
        paths[var]['ctrl'] = [path for path in paths_ctrl if re.search(var+'_', path)][0]
        paths[var]['exp'] = [path for path in paths_exp if re.search(var+'_', path)][0]

    return paths

In [4]:
paths_exp = snakemake.input.experiments_diags + snakemake.input.exp_derived_diag
paths_ctrl = snakemake.input.control_diags + snakemake.input.control_derived_diag

paths = create_dict(paths_ctrl, paths_exp)

areacello = xr.open_dataset(snakemake.input.get('areacello','workflow/input_data/common_grid.nc'))

if "mask" in snakemake.input:
    mask = xr.open_dataset(snakemake.input.mask)
else:
    mask=None

paths_erfs = {path.split('/')[-1].split('_')[0] : path for path in snakemake.input.erfs}
time_slice = snakemake.params.get('time_slice', slice(0,None))
CI_alpha = snakemake.params.get('CI_alpha', 0.05)

In [5]:
diff={}
ctrl={}
exp={}
meta_data = pd.DataFrame(index=['units','description','greater_than_variability',
                                'CI_low','CI_high','CI_alpha','pval', 'tval', 'diff_significant','ctrl_mean', 
                                'exp_mean'],
                         columns=list(paths.keys())+ list(paths_erfs.keys()))
experiment_meta_data = {}
for var, pathdict in paths.items():

    ds_ctrl = xr.open_dataset(pathdict['ctrl']).isel(time=time_slice)
    ds_exp = xr.open_dataset(pathdict['exp']).isel(time=time_slice)

    da_ctrl = ds_ctrl[var]
    da_exp = ds_exp[var]
    units = da_exp.units
    meta_data[var]['units'] = units

    expTs = masked_average(da_exp, mask,areacello['cell_area'], dim=['lon','lat'])
    expTs = expTs.assign_coords(time=expTs.time.dt.year)
    ctrlTs=masked_average(da_ctrl, mask,areacello['cell_area'],dim=['lon','lat'])
    ctrlTs =ctrlTs.assign_coords(time=ctrlTs.time.dt.year)
    diff[var] = expTs-ctrlTs
    ctrl[var] = ctrlTs
    exp[var] = expTs

    meta_data[var]['greater_than_variability'] = diff_means_greater_than_varability(expTs, ctrlTs, mask, areacello['cell_area']).values
    
    CI_low, CI_high, _ = calculate_CI(expTs, ctrlTs, mask=mask, weights=areacello['cell_area'])
    meta_data[var]['CI_low'] = CI_low.values
    meta_data[var]['CI_high'] = CI_high.values
    meta_data[var]['CI_alpha'] = CI_alpha    
    meta_data[var]['description'] = da_exp.long_name
    meta_data[var]['ctrl_mean'] = ctrlTs.mean().values
    meta_data[var]['exp_mean'] = expTs.mean().values
    t_value, p_value, _ = t_test_diff_sample_means(expTs, ctrlTs, mask=mask, weights=areacello['cell_area'])
    meta_data[var]['pval'] = p_value
    meta_data[var]['tval'] = t_value
    meta_data[var]['diff_significant'] = p_value < CI_alpha

    
experiment_meta_data['total_area'] = float(areacello['cell_area'].sum(dim=['lon','lat']))
experiment_meta_data['total_area_units'] = 'm^2'
experiment_meta_data['source_id'] = snakemake.wildcards.model
experiment_meta_data['experiment_id'] = 'piClim-2xdust'
experiment_meta_data['control_id'] = 'piClim-control'
experiment_meta_data['time_slice'] = [time_slice.start, time_slice.stop]
experiment_meta_data['nyear'] = len(ctrlTs)
experiment_meta_data['member_id_ctrl'] = ds_ctrl.attrs['variant_label']
experiment_meta_data['member_id_exp'] = ds_exp.attrs['variant_label']

In [6]:
t0 = next(iter(ctrl))
max_time_dim = [key for key in ctrl if len(ctrl[t0].time) < len(ctrl[key].time)]   
if not max_time_dim:
    max_time_dim = t0
else:
    max_time_dim = max_time_dim[-1]
    
ctrl = {key : make_equal_dimmension(ds, ctrl[max_time_dim]) for key, ds in ctrl.items()}
exp = {key : make_equal_dimmension(ds, exp[max_time_dim]) for key, ds in exp.items()}

ctrl = pd.DataFrame(ctrl)
exp = pd.DataFrame(exp)
exp.to_csv(snakemake.output.exp)
ctrl.to_csv(snakemake.output.ctrl)

In [7]:
for erf, path in paths_erfs.items():
    temp = xr.open_dataset(path)
    meta_data[erf]['units'] = temp[erf].units
    meta_data[erf]['description'] = temp[erf].long_name
    if 'year' in temp.dims:
        temp = temp.rename_dims({'year':'time'})
        temp = temp.rename_vars({'year':'time'})
    temp = temp.isel(time=time_slice).copy()
    temp = masked_average(temp[erf], mask, areacello['cell_area'], dim=['lon','lat'])
    
    diff[erf] = temp
    meta_data[erf]['exp_mean'] = temp.mean().values
    res = bootstrap((temp,),np.mean)
    ci = res.confidence_interval
    meta_data[erf]['CI_low'] = ci.low
    meta_data[erf]['CI_high'] = ci.high
    

In [8]:
t0 = next(iter(diff))
max_time_dim = [key for key in diff if len(diff[t0]) < len(diff[key])]   
if not max_time_dim:
    max_time_dim = t0
else:
    max_time_dim = max_time_dim[-1]
diff = {key : make_equal_dimmension(ds, diff[max_time_dim]) for key, ds in diff.items()}    

In [9]:
diff = pd.DataFrame(diff)
diff.to_csv(snakemake.output.diff)
meta_data.to_csv(snakemake.output.metadata)

with open(snakemake.output.info_yaml, 'w') as of: 
    yaml.safe_dump(experiment_meta_data,of,default_flow_style=False)